# Running BERT Topic on the google drive

The following runs BERT Topic on the google drive


In [2]:
# This initializes pydrive with a set of credentials.  Credential files are not in the git repository - please ask @jintrone for this files in Slack

from pydrive2.auth import GoogleAuth
from pydrive2.drive import GoogleDrive
 
gauth = GoogleAuth()
gauth.LoadCredentialsFile("mycreds.txt")
if gauth.credentials is None:
    gauth.GetFlow()
    gauth.flow.params.update({'access_type': 'offline'})
    gauth.flow.params.update({'approval_prompt': 'force'})
    
    # Authenticate if they're not there
    gauth.LocalWebserverAuth()
elif gauth.access_token_expired:
    # Refresh them if expired
    gauth.Refresh()
else:
    # Initialize the saved creds
    gauth.Authorize()
# Save the current credentials to a file
gauth.SaveCredentialsFile("mycreds.txt")
drive = GoogleDrive(gauth)

In [3]:
# This block just sets up pointers to the relevant directories on the google drive

import re
import pandas as pd
import re

folders = {"CNN":'1Lz8u7wkDr4wYmSzsIKle0lQVWUvO1yim',
"CrooksAndLiars":'102aK5TvKde43bQ-lgS5bX1oVeh1lA6u3',
"NPR": "1ImjUNTH9tzQnQItdjQeb82HppngNx9Ir",
"Reason": "1CaeCbBCpaIsdHtmOa0OhJ-weDKjQqv-P",
"OANN":"1WFCXxukdjngqjQ9HzKVE3f11L2rF9EP4",
          "Demo":"1GwbUCc_k-3_fScEa3bspWdRF0Mpfzjjl"}

download_link = 'https://drive.google.com/uc?export=download&id={}'

driveId = '0AM-VeyaNeDrSUk9PVA'

# Simple function to get a list of files in a given folder
# Use like:  get_files("CNN")

def get_files(folder):
    fileList = drive.ListFile({'q': f"'{folders[folder]}' in parents and trashed=false",'corpora':'drive','driveId':f'{driveId}',"includeItemsFromAllDrives":"true","supportsAllDrives":"true"}).GetList()
    return [f for f in fileList]



In [6]:
# Illustrating how this works with a demo folder I set up

x = get_files("Demo")
pd.DataFrame({"id":i['id'],"name":i['title']} for i in x)

,id,name
0,1zragDSmDe5gEpZE0ptGTDfSDG31-C5En,154041465_CNN.topic_defs.csv
1,1eytMjnGAp0aEANbUXsyXWascjxlbFCIL,154041465_CNN.topics.csv
2,1gyHbAPmcrrAu-vuYQkxybRmh6V23PZB3,154041465_CNN.csv


In [7]:
from bertopic import BERTopic
import os

# Run BERTTopic on a file id - presumes a csv with tweets in a column called "text"
def build_topic_model(id):
    raw = pd.read_csv(download_link.format(id),dtype={'id':str},engine="python")
    raw = raw.dropna(subset=['text'])
    raw.text = raw.apply(lambda row: re.sub(r"http\S+", "", row.text).lower(),1)
    raw = raw.dropna(subset=['text'])
    raw.text = raw.apply(lambda row: " ".join(filter(lambda x:x[0]!="@", row.text.split())), 1)
    raw = raw.dropna(subset=['text'])
    raw.text = raw.apply(lambda row: " ".join(re.sub("[^a-zA-Z]+", " ", row.text).split()), 1)
    raw = raw.loc[(raw.text != ""), :]
    timestamps = raw.created_at.to_list()
    tweets = raw.text.to_list()
    
    topic_model = BERTopic(verbose=True)
    topics, probs = topic_model.fit_transform(tweets)
    topic_df = pd.DataFrame({"id":raw.id.tolist(),"topic":topics})
    topic_df = topic_df.loc[(topic_df.topic!=-1),:]
    topic_df['weight']=1
    topic_defs = pd.DataFrame.from_dict(topic_model.get_topics(),orient="index").reset_index().melt(id_vars=['index']).sort_values(['index','variable'])
    topic_defs = pd.concat([topic_defs["index"].reset_index(drop=True),pd.DataFrame(topic_defs['value'].tolist(),columns=["word","weight"]).reset_index(drop=True)],axis=1).rename(columns={"index":"topic"})
    return pd.pivot_table(topic_df,values = 'weight', fill_value=0, columns = 'topic',index = 'id'), topic_defs

# Upload single data file to a drive
def upload_data_file(folderid,filename,data,id=None):
    data.to_csv(filename)
    meta = {"mimeType": "text/csv", "parents": [{"id": folderid}]}
    if id:
        meta["id"] = id
    file1 = drive.CreateFile(meta)
    file1.SetContentFile(filename)
    file1.Upload(param={'supportsTeamDrives': True}) # Upload the file.
    os.remove(filename)

# Upload the processed topic model / defs to a drive
def upload_files(stub,folder,topics,defs,ids):
    fnames = {f"{stub}.topics.csv":topics,f"{stub}.topic_defs.csv":defs}
    upload_data_file(folders[folder],f"{stub}.topics.csv",topics,ids.get("topic"))
    upload_data_file(folders[folder],f"{stub}.topic_defs.csv",defs,ids.get("topic_defs"))
        

# Generates a list of raw files to process - if overwrite is True, this will also 
# replace existing files      
def get_files_to_process(folder,overwrite = False):
    f = get_files(folder)
    reject = set()
    retain = {}
    for i in f:
        name = i['title']
        if not name.endswith(".csv") or "annotation" in name:
            continue
        stub = name.split(".")[0]
        if stub not in retain:
            retain[stub] = {}
        
        if "topic_defs" in name:
            retain[stub]['topic_defs'] = i['id']
        elif "topic" in name:
            retain[stub]['topic'] = i['id']
        else:
            retain[stub]['main'] = i['id']
    
    if not overwrite:
        retain = { k:v for k,v in retain.items() if 'topic' not in v }
    return retain

# One function to rule them all
def process_folder(folder,overwrite = False):
    files = get_files_to_process(folder,overwrite)
    for f,i in files.items():
        print(f"Processing {f}:{i}")
        topics, defs = build_topic_model(i['main'])
        upload_files(f,folder,topics,defs,i)


/home/jeintron/dev/topic_modeling/bert_topic/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
test = '125jKjLQFldQUPPkWAk94WHbcffFlBlon'



In [8]:
process_folder("Reason")


Processing 102155783_Reason:{'main': '10DP8bE3xeor9QcRksVW6Q_Fbjr0GkS0B'}


Batches: 100%|██████████| 22/22 [00:00<00:00, 119.44it/s]
2022-10-14 15:36:33,973 - BERTopic - Transformed documents to Embeddings
2022-10-14 15:36:40,408 - BERTopic - Reduced dimensionality
2022-10-14 15:36:40,445 - BERTopic - Clustered reduced embeddings


Processing 275757377_Reason:{'main': '1Z8XduQCuWP2qenfCvJmyI_ao52rnD0_2'}


Batches: 100%|██████████| 97/97 [00:00<00:00, 131.58it/s]
2022-10-14 15:36:46,579 - BERTopic - Transformed documents to Embeddings
2022-10-14 15:36:56,882 - BERTopic - Reduced dimensionality
2022-10-14 15:36:57,060 - BERTopic - Clustered reduced embeddings


Processing 398960000_Reason:{'main': '1G9HNvtCZrDYKsjoe3_J8eZrhXHgyyHVc'}


...

Batches: 100%|██████████| 146/146 [00:01<00:00, 109.21it/s]
2022-10-14 15:46:46,011 - BERTopic - Transformed documents to Embeddings
2022-10-14 15:46:58,860 - BERTopic - Reduced dimensionality
2022-10-14 15:46:58,988 - BERTopic - Clustered reduced embeddings


Processing 106863668_Reason:{'main': '153MTbTZJ0PTq27bqvF4JZjpKYwJiXh2B'}


Batches: 100%|██████████| 1393/1393 [00:10<00:00, 133.03it/s]
2022-10-14 15:47:20,323 - BERTopic - Transformed documents to Embeddings
2022-10-14 15:47:49,518 - BERTopic - Reduced dimensionality
2022-10-14 15:47:52,667 - BERTopic - Clustered reduced embeddings


Processing 14352000_Reason:{'main': '1FBr4fyPfhytSfFrkehEz_id7aP6v7cvR'}


HTTPError: HTTP Error 403: Forbidden